In [48]:
import numpy as np
import plotly.express as px
from visualizations.data import viz_data_prep as vdp

In [53]:
input_params = {
    "k": 115740740740,
    "lambda": 750000000000000000,
    "delta": 2475000000000000,
    "capPayoff": 5000000000000000000,
    "capNotional": 20000000000000000000000,
    "capLeverage": 10000000000000000000,
    "circuitBreakerWindow": 2592000,
    "circuitBreakerMintTarget": 1666666666666666666666,
    "maintenanceMarginFraction": 40000000000000000,
    "maintenanceMarginBurnRate": 50000000000000000,
    "liquidationFeeRate": 50000000000000000,
    "tradingFeeRate": 750000000000000,
    "minCollateral": 100000000000000,
    "priceDriftUpperLimit": 87000000000000,
    "averageBlockTime": 0
}

## Explanations for formula based features

In [29]:
ask_delta = vdp.ask(1, input_params['delta']/1e18, 0, 0)
bid_delta = vdp.bid(1, input_params['delta']/1e18, 0, 0)

ask_impact = vdp.ask(1, 0, input_params['lambda']/1e18, 0.01)
print(f'''
Funding rate per day: {round(vdp.funding_per_sec(input_params['k']/1e18) * 86400 * 100, 2)}%
Static bid-ask spread: {round(100*((ask_delta/bid_delta) - 1), 2)}%
Market Impact (for trade with size 1% of OI cap): {round((vdp.ask(1, 0, input_params['lambda']/1e18, 0.01) - 1) * 100, 2)}%
Notional cap as percentage of 8mn OVL: {round(input_params['capNotional'] * 100/(1e18 * 8_000_000), 2)}%
Max allowed price change of longer TWAP: {round(vdp.price_drift_allowed(input_params['priceDriftUpperLimit']/1e18, 3600) * 100, 2)}%
Payoffs capped at: {input_params['capPayoff']/1e18} times input collateral
''')


Funding rate per day: 2.0%
Static bid-ask spread: 0.5%
Market Impact (for trade with size 1% of OI cap): 0.75%
Notional cap as percentage of 8mn OVL: 0.25%
Max allowed price change of longer TWAP: 36.78%
Payoffs capped at: 5.0 times input collateral



In [52]:
base_price = 1
ask_list = []
bid_list = []
oi_perc_range = np.arange(0, 101, 1)
for oi_perc in oi_perc_range:
    bid_ask_inputs = (
        base_price,
        input_params['delta']/1e18,
        input_params['lambda']/1e18,
        oi_perc/100
    )
    ask_list.append(
        vdp.ask(*bid_ask_inputs)
    )
    bid_list.append(
        vdp.bid(*bid_ask_inputs)
    )

ask_perc = [(ask - 1) * 100 for ask in ask_list]
bid_perc = [(bid - 1) * 100 for bid in bid_list]
notional_ovl = [(oi_perc/100) * input_params['capNotional']/1e18 for oi_perc in oi_perc_range]

# Plot ask_perc vs oi_perc
fig_ask = px.line(
    x=notional_ovl, y=ask_perc,
    labels={'x': 'Notional OVL', 'y': 'Market Impact (%, including static spread)'},
    title='Market Impact (for asks) vs Trade Size'
)
fig_ask.show()

# Plot bid_perc vs oi_perc
fig_bid = px.line(
    x=notional_ovl, y=bid_perc,
    labels={'x': 'Notional OVL', 'y': 'Market Impact (%, including static spread)'},
    title='Market Impact (for bids) vs Trade Size'
)
fig_bid.show()